# Exploring and reproducing periodic table properties

2024-09-14

In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

pd.options.plotting.backend = "plotly"

## Ionisation energies

In [2]:
df = pd.read_csv('table_ionisation.csv')

In [3]:
df.head()

,Element,Symbol,Atomic Number,Mass Number,Ionization Energy (eV)
0,Hydrogen,H,1,1,13.598
1,Helium,He,2,4,24.587
2,Lithium,Li,3,7,5.392
3,Beryllium,Be,4,9,9.322
4,Boron,B,5,11,8.298


In [10]:
# levels = [1*2, 1*(1+2)*2, 1*(1+2*2)*2]
levels = [(1+2*i)*2 for i in range(5)]

levels

[2, 6, 10, 14, 18]

In [8]:
[sum(levels[:i]) for i in range(1, 10)]

[2, 8, 18, 32, 50, 72, 98, 128, 162]

In [17]:
# shells = [2, 2, 6, 2, 6, 2, 10, 6, 2, 10, 6, 2, 14, 10, 6, 2]  # etc
shells = [list(range(2, n_level, 4))[::-1] for n_level in levels[1:]]
shells

In [20]:
shells_flat = []
for shell in shells:
    shells_flat.extend(shell)
    shells_flat.extend(shell)

shells_flat

[2, 2, 6, 2, 6, 2, 10, 6, 2, 10, 6, 2, 14, 10, 6, 2, 14, 10, 6, 2]

In [38]:
shells_cumul = np.cumsum(shells_flat)
shells_cumul = np.insert(shells_cumul, 0, 0)

shells_cumul

array([  0,   2,   4,  10,  12,  18,  20,  30,  36,  38,  48,  54,  56,
        70,  80,  86,  88, 102, 112, 118, 120])

In [54]:
# example of valence electrons for atom with 33 electrons

num_e = 3
idx = ((num_e - shells_cumul) > 0).sum()
num_e - shells_cumul[idx-1]  # number of valence electrons

1

In [56]:
def compute_valence_electrons(num):
    '''Function to compute the number of valence electrons in an atom given atomic number'''
    if num <= 2:
        return num
    elif num <= 10:
        return num - 2
    elif num <= 18:
        return num - 10
    elif num <= 36:
        return num - 18
    elif num <= 54:
        return num - 36
    elif num <= 86:
        return num - 54
    else:
        return num - 86
    

def compute_valence_electrons(num_e):
    idx = ((num_e - shells_cumul) > 0).sum()
    val_e = num_e - shells_cumul[idx-1]
    return val_e


def compute_inside_electrons(num_e):
    idx = ((num_e - shells_cumul) > 0).sum()
    return shells_cumul[idx-1]

### Data analysis

In [60]:
# compute inside electrons
df['inside electrons'] = df['Atomic Number'].map(lambda x: compute_inside_electrons(x))
df['valence eletrons'] = df['Atomic Number'].map(lambda x: compute_valence_electrons(x))

In [61]:
# compute effective charge
df['Z1'] = df['Atomic Number'] - df['inside electrons']
df['Z2'] = df['Atomic Number'] - df['inside electrons'] - (df['valence eletrons']-1) / 2

In [64]:
df.head(13)

,Element,Symbol,Atomic Number,Mass Number,Ionization Energy (eV),inside electrons,valence eletrons,Z1,Z2
0,Hydrogen,H,1,1,13.598,0,1,1,1.0
1,Helium,He,2,4,24.587,0,2,2,1.5
2,Lithium,Li,3,7,5.392,2,1,1,1.0
3,Beryllium,Be,4,9,9.322,2,2,2,1.5
4,Boron,B,5,11,8.298,4,1,1,1.0
5,Carbon,C,6,12,11.260,4,2,2,1.5
6,Nitrogen,N,7,14,14.534,4,3,3,2.0
7,Oxygen,O,8,16,13.618,4,4,4,2.5
8,Fluorine,F,9,19,17.423,4,5,5,3.0
9,Neon,Ne,10,20,21.565,4,6,6,3.5


In [85]:
fig = px.line(x=df['Atomic Number'], y=df['Ionization Energy (eV)'], markers=True, text=df['Symbol'])
fig.update_traces(textposition="bottom right")
fig.show()

In [82]:
px.line(x=df['Atomic Number'], y=df['Z2'], markers=True)